## Lab 7: Eel Distribution Modeling with XGBoost

Author: Zoe Zhou

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")



### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Assign target and feature variables
y = model_data.Angaus
X = model_data.drop('Angaus', axis=1)
# X.dtypes # Check column data types

# Encode categorical feature
encoder = LabelEncoder()
X.Method = encoder.fit_transform(X.Method)

# Split into training and validation (not testing!)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 808)



### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [3]:
# Build XGB model with defined hyperparameters
xgb = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    eval_metric="logloss",
    early_stopping_rounds=50,
    random_state=808)

# fit model with training data and validation sets
xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Print best n_estimators from xgb model 
best_n_estimators = xgb.best_iteration
print(f"Best number of trees in baseline model is {best_n_estimators}")

[0]	validation_0-logloss:0.64173
[1]	validation_0-logloss:0.59874
[2]	validation_0-logloss:0.56352
[3]	validation_0-logloss:0.53716
[4]	validation_0-logloss:0.51702
[5]	validation_0-logloss:0.50016
[6]	validation_0-logloss:0.48135
[7]	validation_0-logloss:0.46672
[8]	validation_0-logloss:0.45634
[9]	validation_0-logloss:0.44405
[10]	validation_0-logloss:0.43728
[11]	validation_0-logloss:0.42904
[12]	validation_0-logloss:0.41955
[13]	validation_0-logloss:0.41371
[14]	validation_0-logloss:0.40708
[15]	validation_0-logloss:0.40332
[16]	validation_0-logloss:0.39853
[17]	validation_0-logloss:0.39618
[18]	validation_0-logloss:0.39130
[19]	validation_0-logloss:0.38952
[20]	validation_0-logloss:0.38569
[21]	validation_0-logloss:0.38372
[22]	validation_0-logloss:0.37811
[23]	validation_0-logloss:0.37809
[24]	validation_0-logloss:0.37631
[25]	validation_0-logloss:0.37448
[26]	validation_0-logloss:0.37215
[27]	validation_0-logloss:0.37295
[28]	validation_0-logloss:0.37361
[29]	validation_0-loglos

### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [4]:
# Initialize a new model
xgb_lr = XGBClassifier(
    eval_metric='logloss', 
    random_state=808,
    early_stopping_rounds=50,
    n_estimators=best_n_estimators
    )
# fit model with training data and validation sets
#xgb_lr.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Define hyperparameter distribution
param_dist = {"learning_rate": uniform(0.01, 0.31)}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    xgb_lr, param_dist, n_iter =20, scoring="accuracy",
    cv=3, verbose=1, random_state=808)

# Run random search
random_search.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Get search results as a DataFrame
results = pd.DataFrame(random_search.cv_results_)

# Print best learning rate
best_lr = random_search.best_params_['learning_rate']
print(f"Best learning rate: {best_lr: .4f}")

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[0]	validation_0-logloss:0.56598
[1]	validation_0-logloss:0.49544
[2]	validation_0-logloss:0.45688
[3]	validation_0-logloss:0.42429
[4]	validation_0-logloss:0.40567
[5]	validation_0-logloss:0.39485
[6]	validation_0-logloss:0.38229
[7]	validation_0-logloss:0.38562
[8]	validation_0-logloss:0.38670
[9]	validation_0-logloss:0.39327
[10]	validation_0-logloss:0.38495
[11]	validation_0-logloss:0.39029
[12]	validation_0-logloss:0.38769
[13]	validation_0-logloss:0.39084
[14]	validation_0-logloss:0.39198
[15]	validation_0-logloss:0.39594
[16]	validation_0-logloss:0.39759
[17]	validation_0-logloss:0.40102
[18]	validation_0-logloss:0.39725
[19]	validation_0-logloss:0.40199
[20]	validation_0-logloss:0.40654
[21]	validation_0-logloss:0.40788
[22]	validation_0-logloss:0.40541
[23]	validation_0-logloss:0.40477
[24]	validation_0-logloss:0.40656
[25]	validation_0-logloss:0.40472
[26]	validation_0-logloss:0.40612
[27]	validation_0-logloss:0.408

### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [ ]:
print(f"Best learning rate: {best_lr: .4f}")
print(f"Best number of trees in baseline model is {best_n_estimators}")

In [ ]:
# Initialize your model with the best_n_estinators & best_lr
xgb_tree = XGBClassifier(
    n_estimators=best_n_estimators,
    learning_rate=best_lr,
    early_stopping_rounds=50,
    eval_metric="logloss",
    random_state=808)

# Set param distribution
param_dist_tree = {
    "max_depth": randint(3, 10),
    "min_child_weight": randint(1, 10),
    "gamma": uniform(0.05, 0.1)
}

# Set up RandomizedSearchCV
random_search_tree = RandomizedSearchCV(
    xgb_tree, 
    param_dist_tree,
    n_iter=20,
    scoring="accuracy",
    cv=3,
    verbose=False,
    random_state=808
    )

# Fit CV model 
random_search_tree.fit(X_train, y_train, eval_set=[(X_val, y_val)])

# Print best tree hyperparameters
best_tree_params = random_search_tree.best_params_
print(f"Best tree parameters: {best_tree_params}")
print(f"Best learning rate: {best_lr: .4f}")
print(f"Best number of trees in baseline model is {best_n_estimators}")


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and 1 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and 1
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [ ]:
# Initialize model for tuning stochastic components
xgb_sc = XGBClassifier(
    early_stopping_rounds=50,
    eval_metric="logloss",
    random_state=808,
    n_estimators=best_n_estimators,
    learning_rate=best_lr,
    **best_tree_params,
    )

# Define param dist within stochastic component 
param_dist_sc = {
    "subsample":uniform(0.5,1),
    "colsample_bytree":uniform(0.5,1)
}

# Initialize CV model
random_search_sc = RandomizedSearchCV(
    xgb_sc, param_dist_sc, n_iter=20, scoring="accuracy",
    cv=3, verbose=False, random_state=808
)

# Fit search
random_search_sc.fit(X_train, y_train, eval_set=[(X_val, y_val)])

In [ ]:
# print best stochastic params
best_sc = random_search_sc.best_params_
print(f"Best stochastic components: {best_sc}")
print(f"Best tree parameters: {best_tree_params}")
print(f"Best learning rate: {best_lr: .4f}")
print(f"Best number of trees in baseline model is {best_n_estimators}")


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [ ]:
# Prepare evaluation data
X_eval = eval_data.drop("Angaus_obs", axis=1)
y_eval = eval_data['Angaus_obs']

# Encode Method column
X_eval.Method = encoder.fit_transform(X_eval.Method)

# Initialize final model
xgb_final = XGBClassifier(
    n_estimators=best_n_estimators,
    learning_rate=best_lr,
    **best_tree_params,
    **best_sc,
    early_stopping_rounds=50,
    eval_metric="logloss",
    random_state=808
)

# Fit model
xgb_final.fit(X_train, y_train, eval_set=[(X_val, y_val)])
xgb_preds = xgb_final.predict(X_eval)

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [ ]:
# Calculate predition probability
xgb_proba = xgb_final.predict_proba(X_eval)[:,1]

# Compute AUC
auc_score = roc_auc_score(y_eval, xgb_proba)
print(f"AUC Score: {auc_score:.4f}")

In [ ]:
# Obtain feature importance from final model
imp = pd.DataFrame({'Feature': X_train.columns, 'Importance':xgb_final.feature_importances_})

# Sort and print feature importance
feature_imp = imp.sort_values(by="Importance", ascending=False)
feature_imp

### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

My model achieved an AUC score of 0.862, which is slightly lower than the cross-validation model in Elith et al. that obtained an AUC of 0.869 . This indicates that Elith et al.'s model has a marginal advantage in predictive performance. Both models identified similar top three important features: `SegSumT`, `Method`, `USNative`. However, in my model, `Method` ranked second instead of third as in Elith et al.'s model. Another way to compare the models can be interpretability and computational efficiency. But it's hard to say which model performed better without measuring appropriate metrics. 